In [1]:
import pandas as pd
import json
import ast
import re
from constants import * 

In [2]:
from constants import * 

# создаем файл для категорий второго уровня
res = {i: dict(enumerate(values)) for i, values in enumerate(FIRST_LEVEL.values())}

with open('zeroshot/data/zeroshot_topics_2.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)

## Обработка

In [3]:
df = pd.read_csv('data/final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4493 entries, 0 to 4492
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ссылка на статью     4493 non-null   object 
 1   DOI                  4076 non-null   object 
 2   eLIBRARY ID          4493 non-null   int64  
 3   article_type         4493 non-null   object 
 4   year                 4493 non-null   int64  
 5   language             4493 non-null   object 
 6   title_en             4274 non-null   object 
 7   key_words_en         4232 non-null   object 
 8   abstract_en          4172 non-null   object 
 9   authors_metadata_en  4485 non-null   object 
 10  title_ru             1757 non-null   object 
 11  key_words_ru         1757 non-null   object 
 12  abstract_ru          1717 non-null   object 
 13  authors_metadata_ru  1757 non-null   object 
 14  OECD                 4493 non-null   object 
 15  Название журнала     3632 non-null   o

In [5]:
# исправляем 'русский, английский' на 'английски' и оставляем только русский и английский
df.loc[df.language == 'русский, английский', 'language'] = 'английский'
df = df[df.language.isin(['английский', 'русский'])]
df.language.value_counts()

#df.to_csv('data/final_lang_clean', index=False)

language
английский    2768
русский       1702
Name: count, dtype: int64

In [6]:
# удаляем строки title_en = None, abstract_en = None, a key_words_en = None запоняем '[]'
df = df.dropna(subset=['title_en', 'abstract_en'])
df.loc[df.key_words_en.isna(), 'key_words_en'] = '[]'
print(f'Ключевые слова отсуствуют для {df[df.key_words_en == '[]'].shape[0]} статей\n')
df.info()

df.to_csv('zeroshot/data/final_clean.csv', index=False)

Ключевые слова отсуствуют для 787 статей

<class 'pandas.core.frame.DataFrame'>
Index: 4165 entries, 0 to 4492
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Ссылка на статью     4165 non-null   object 
 1   DOI                  3847 non-null   object 
 2   eLIBRARY ID          4165 non-null   int64  
 3   article_type         4165 non-null   object 
 4   year                 4165 non-null   int64  
 5   language             4165 non-null   object 
 6   title_en             4165 non-null   object 
 7   key_words_en         4165 non-null   object 
 8   abstract_en          4165 non-null   object 
 9   authors_metadata_en  4157 non-null   object 
 10  title_ru             1498 non-null   object 
 11  key_words_ru         1498 non-null   object 
 12  abstract_ru          1498 non-null   object 
 13  authors_metadata_ru  1498 non-null   object 
 14  OECD                 4165 non-null   object 
 15  Н

## Создание json

In [12]:
# очистка текста
def clean_text(text):
    if pd.isna(text):
        return ""
    text = re.sub(r'Unknown node type: \w+', '', text)
    text = ' '.join(text.split())
    return text

# keywords в список
def parse_keywords(kw_str):
    if pd.isna(kw_str) or kw_str.strip() == "":
        return []
    try:
        return ast.literal_eval(kw_str)
    except Exception:
        return [kw_str]
    
# функция для создания правльного json
def prepare_json(df, filename, root, columns_rename=COLUMNS_RENAME):
    df_json = df[list(columns_rename.keys())].rename(columns=columns_rename)
    df_json['abstract'] = df_json['abstract'].apply(clean_text)
    df_json['keywords'] = df_json['keywords'].apply(parse_keywords)

    records = df_json.to_dict(orient='records')
    print(f'Размер {root}{filename} df: {df_json.shape[0]}')

    with open(f'{root}{filename}.json', 'w', encoding='utf-8') as f:
        json.dump(records, f, ensure_ascii=False, indent=4)
    
    return f'{root}{filename}.json успешно сохранен' 

In [13]:
df = pd.read_csv('zeroshot/data/final_clean.csv')

df_train = df[df['first_OECD'] == 'нет'].copy()
df_test  = df[df['first_OECD'] != 'нет'].copy()

print(prepare_json(df_train, filename='train', root='zeroshot/data/train/'))
print(prepare_json(df_test, filename='test', root='zeroshot/data/test/'))

Размер zeroshot/data/train/train df: 2859
zeroshot/data/train/train.json успешно сохранен
Размер zeroshot/data/test/test df: 1306
zeroshot/data/test/test.json успешно сохранен


In [14]:
#df = pd.read_csv('')
df = pd.read_csv('zeroshot/data/final_clean.csv')
for i, k in enumerate(FIRST_LEVEL.keys()): 

    df_train = df[(df['first_OECD'] == k) & (df['OECD'] == 'нет')].copy()
    df_test  = df[df['first_OECD'] == k].copy()

    print(prepare_json(df_train, filename=f'{i}', root='zeroshot/data/train/'))
    print(prepare_json(df_test, filename=f'{i}', root='zeroshot/data/test/'))


Размер zeroshot/data/train/0 df: 0
zeroshot/data/train/0.json успешно сохранен
Размер zeroshot/data/test/0 df: 222
zeroshot/data/test/0.json успешно сохранен
Размер zeroshot/data/train/1 df: 0
zeroshot/data/train/1.json успешно сохранен
Размер zeroshot/data/test/1 df: 79
zeroshot/data/test/1.json успешно сохранен
Размер zeroshot/data/train/2 df: 0
zeroshot/data/train/2.json успешно сохранен
Размер zeroshot/data/test/2 df: 52
zeroshot/data/test/2.json успешно сохранен
Размер zeroshot/data/train/3 df: 0
zeroshot/data/train/3.json успешно сохранен
Размер zeroshot/data/test/3 df: 9
zeroshot/data/test/3.json успешно сохранен
Размер zeroshot/data/train/4 df: 0
zeroshot/data/train/4.json успешно сохранен
Размер zeroshot/data/test/4 df: 840
zeroshot/data/test/4.json успешно сохранен
Размер zeroshot/data/train/5 df: 0
zeroshot/data/train/5.json успешно сохранен
Размер zeroshot/data/test/5 df: 104
zeroshot/data/test/5.json успешно сохранен
